In [1]:
%load_ext rich

In [2]:
import math

import phoenix as px

import promptimus as pm

In [3]:
provider = pm.llms.OpenAILike(
    model_name="gemma3:12b",
    base_url="http://lilan:11434/v1",
    api_key="DUMMY",
)

In [4]:
def power(a: float, b: float) -> float:
    """Calcuates the `a` in the pover of `b`"""
    return a**b


def factorial(a: int) -> int:
    """Calcuates the factorial (!) of `a`"""
    return math.factorial(a)


def multiply(a: float, b: float) -> float:
    """Multiplies `a` and `b`"""
    return a * b

In [5]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
tool_calling = pm.modules.ToolCallingAgent(
    [power, multiply, factorial], observation_role=pm.MessageRole.USER
).with_provider(provider)
pm.trace_phoenix(
    tool_calling,
    "agent",
    project_name="tool_calling_test",
    endpoint="http://localhost:4317",
)

2025-04-22 11:40:00.924 | DEBUG    | promptimus.tracing.phoenix:_trace_module:150 - Wrapping `agent.tools`
2025-04-22 11:40:00.925 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.power`
2025-04-22 11:40:00.925 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.multiply`
2025-04-22 11:40:00.926 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.factorial`
2025-04-22 11:40:00.927 | DEBUG    | promptimus.tracing.phoenix:_trace_module:150 - Wrapping `agent.predictor`
2025-04-22 11:40:00.927 | DEBUG    | promptimus.tracing.phoenix:_trace_module:140 - Wrapping `agent.predictor.prompt`


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: tool_calling_test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [7]:
await tool_calling.forward("What is twice the factorial of 3?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='12.0', tool_calls=None, tool_call_id=None)

![](assets/phoenix_example.jpg)

In [8]:
openai = pm.llms.OpenAILike(
    model_name="gpt-4.1-nano",
)

tool_calling = pm.modules.OpenaiToolCallingAgent(
    "Utilize your tools step by step, to act as a calculator.",
    [power, multiply, factorial],
).with_provider(openai)

pm.trace_phoenix(
    tool_calling,
    "agent",
    project_name="openai_tool_calling_test",
    endpoint="http://localhost:4317",
)

2025-04-22 11:40:03.485 | DEBUG    | promptimus.tracing.phoenix:_trace_module:150 - Wrapping `agent.tools`
2025-04-22 11:40:03.485 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.power`
2025-04-22 11:40:03.486 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.multiply`
2025-04-22 11:40:03.486 | DEBUG    | promptimus.tracing.phoenix:_trace_module:145 - Wrapping `agent.tools.factorial`
2025-04-22 11:40:03.486 | DEBUG    | promptimus.tracing.phoenix:_trace_module:150 - Wrapping `agent.predictor`
2025-04-22 11:40:03.487 | DEBUG    | promptimus.tracing.phoenix:_trace_module:140 - Wrapping `agent.predictor.prompt`


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: openai_tool_calling_test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [9]:
await tool_calling.forward("What is twice the factorial of 3?")

/home/ailadin/projects/promptimus/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'inspect._empty'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)



Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='The factorial of 3 is 6, and twice that value is 12.',
    tool_calls=None,
    tool_call_id=None
)

In [10]:
await tool_calling.forward("What is twice the factorial of 3?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='The factorial of 3 is 6, and twice that value is 12.',
    tool_calls=None,
    tool_call_id=None
)